In [1]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

import copy
import random

In [2]:
def select_important_feautre(columns, feature_importance, num):
    
    #딕셔너리
    dic = {}
    for value in [list(v) for v in zip(columns, feature_importance)]:
        dic[value[1]] = value[0] 
        
    sorted_list = sorted(dic)
    sorted_list.reverse()
    
    i = 0
    result = []
    for y in sorted_list:
        result.append(dic[y])
        i = i+1
        if(i == num):
            break
    
    return result

## 데이터셋

In [3]:
x_train_load_fin_v2 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_final_v2.csv').sort_values(by='acc_id', ascending=True)
y_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_label.csv').sort_values(by='acc_id', ascending=True)

In [4]:
x_train_fin_v2 = x_train_load_fin_v2
x_train_fin_v2 = x_train_load_fin_v2.drop(['acc_id'], axis=1)
y_data = y_train_load.label

## 스케일링

In [5]:
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [6]:
mm = pd.DataFrame(minmax_scale(x_train_fin_v2), columns=x_train_fin_v2.columns)

In [7]:
rb = pd.DataFrame(robust_scale(x_train_fin_v2), columns=x_train_fin_v2.columns)

In [8]:
ma = pd.DataFrame(maxabs_scale(x_train_fin_v2), columns=x_train_fin_v2.columns)

## 다양한 모델들

In [28]:
def cross_val_score_custom(model, x, y, cv, dev = 0):
    
    print('\n\n')
    
    result = []
    
    for train_idx, test_idx in cv.split(x):
        
        new_x_train = x.iloc[train_idx, :]
        new_y_train = y.iloc[train_idx]
        
        new_x_test = x.iloc[test_idx, :]
        new_y_test = y.iloc[test_idx]
        
        model.fit(new_x_train, new_y_train)
        
        temp = np.mean(f1_score(new_y_test, model.predict(new_x_test), average=None))
        
        if dev:
            print('dev Mode # : {}'.format(temp))
        
        result.append(temp)
        
        break
    
    print('\n\n')
    
    return np.mean(result)

### 로지스틱 회귀

In [29]:
from sklearn.linear_model import LogisticRegression

In [36]:
logistic_lr = LogisticRegression()

In [37]:
cross_val_score_custom(logistic_lr, mm, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6489459070368978





0.6489459070368978

In [49]:
cross_val_score_custom(logistic_lr, rb, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.3537654600733821





0.3537654600733821

In [50]:
cross_val_score_custom(logistic_lr, ma, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6488679572157476





0.6488679572157476

### 소프트맥스 회귀

In [42]:
soft_lr = LogisticRegression(multi_class="multinomial", solver="lbfgs", C = 10)

In [43]:
cross_val_score_custom(soft_lr, mm, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6542669658009741





0.6542669658009741

In [51]:
cross_val_score_custom(soft_lr, rb, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.3889284568540716





0.3889284568540716

In [52]:
cross_val_score_custom(soft_lr, ma, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6530358669068957





0.6530358669068957

### 서포트 벡터 머신

##### 선형 svm

In [44]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [45]:
linear_svm = LinearSVC()

In [46]:
cross_val_score_custom(linear_svm, mm, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6430338047177713





0.6430338047177713

In [53]:
cross_val_score_custom(linear_svm, rb, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.29985335988363854





0.29985335988363854

In [54]:
cross_val_score_custom(linear_svm, ma, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6430338047177713





0.6430338047177713

##### 커널 svm

In [48]:
kernel_svm = SVC(kernel='rbf', C=10, gamma=1)

In [55]:
cross_val_score_custom(kernel_svm, mm, y_data, KFold(n_splits=5), dev = 1)




dev Mode # : 0.6989889369379778





0.6989889369379778

In [ ]:
cross_val_score_custom(kernel_svm, rb, y_data, KFold(n_splits=5), dev = 1)

In [ ]:
cross_val_score_custom(kernel_svm, ma, y_data, KFold(n_splits=5), dev = 1)

### 부스팅 계열 알고리즘